# Exercise 2.2 Solution - Computing Stress Fields with Tensors

### Task
Estimate the mass density given a displacement field assuming a gravity load

### Learning goals: 
To familiarize yourself with the PyTorch framework with an example from mechanics


In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.autograd import grad

displacements

In [ ]:
u = lambda x, y: 0 * x ** 2 * y ** 2  # needed to force pytorch to evaluate gradient
v = lambda x, y: -1.7004e-7 * y ** 2 + x ** 2 * y ** 2 * 0

definition of a spatial grid

In [ ]:
nx = 5
ny = 5

x = torch.linspace(-1, 1, nx, requires_grad=True)
y = torch.linspace(-1, 1, ny, requires_grad=True)
x, y = torch.meshgrid(x, y, indexing="ij")

sampled displacement field

```
d.size = ([2, 5, 5])
d[0,:,:] = u(x, y)
d[1,:,:] = v(x, y)
```

In [ ]:
d = torch.cat((u(x, y).unsqueeze(0), v(x, y).unsqueeze(0)), 0)

Visualize displacement field

In [ ]:
plt.figure(figsize=(4, 3))
plt.quiver(x.detach(), y.detach(), d[0, :, :].detach(), d[1, :, :].detach())
plt.xlabel('x')
plt.ylabel('y')
plt.title('Quiver Plot of Vector Field (u, v)')
plt.grid(True)
plt.show()

gradient computation

In [ ]:
dd_dx = torch.zeros((2, 2, nx, ny))
dd_dx[0, 0] = grad(d[0], x, torch.ones_like(x), create_graph=True, retain_graph=True)[0]  # dd_dxdx
dd_dx[0, 1] = grad(d[0], y, torch.ones_like(y), create_graph=True, retain_graph=True)[0]  # dd_dxdy
dd_dx[1, 0] = grad(d[1], x, torch.ones_like(x), create_graph=True, retain_graph=True)[0]  # dd_dydx
dd_dx[1, 1] = grad(d[1], y, torch.ones_like(y), create_graph=True, retain_graph=True)[0]  # dd_dydy

In [ ]:
print(dd_dx[0, 0],
      dd_dx[0, 1],
      dd_dx[1, 0],
      dd_dx[1, 1])

strain computation

In [ ]:
eps = 0.5 * (dd_dx + dd_dx.permute((1, 0, 2, 3)))

material properties and 4th order material tensor

In [ ]:
E = 210000.0
nu = 0.3

C = torch.zeros((2, 2, 2, 2))  # 4th order material tensor
C[0, 0, 0, 0] = 1.0
C[0, 0, 1, 1] = nu
C[1, 1, 0, 0] = nu
C[1, 1, 1, 1] = 1.0
C[0, 1, 0, 1] = (1.0 - nu) / 2.0
C = E / (1.0 - nu ** 2) * C

stress computation

In [ ]:
sig = torch.tensordot(C, eps)

equilibrium computation

In [ ]:
grad(
    sig[0, 0], x, torch.ones_like(x), create_graph=True, retain_graph=True, allow_unused=True
)

In [ ]:
dsig11_dx = grad(
    sig[0, 0], x, torch.ones_like(x), create_graph=True, retain_graph=True, allow_unused=True
)[0]
dsig12_dy = grad(
    sig[0, 1], y, torch.ones_like(y), create_graph=True, retain_graph=True, allow_unused=True
)[0]
dsig21_dx = grad(
    sig[1, 0], x, torch.ones_like(x), create_graph=True, retain_graph=True, allow_unused=True
)[0]
dsig22_dy = grad(
    sig[1, 1], y, torch.ones_like(y), create_graph=True, retain_graph=True, allow_unused=True
)[0]

f = torch.zeros((2, nx, ny))
f[0] = -dsig11_dx - dsig12_dy  # out of balance force in x1
f[1] = -dsig21_dx - dsig22_dy  # out of balance force in x2

In [ ]:
grad(sig[1, 0], x, torch.ones_like(x), create_graph=True, retain_graph=True, allow_unused=True)

density computation

In [ ]:
g = 9810
rho = f[1] / g
print(rho)

### Result Visualization: Density

**Note:** though there is only one color, the value represented is not constant 

In [ ]:
fig, ax = plt.subplots()
cp = ax.contourf(x.detach(), y.detach(), rho.detach(), levels=12, cmap=plt.cm.jet)
fig.colorbar(cp, format="%.2e")
ax.set_aspect("equal")
fig.tight_layout()
plt.show()